In [1]:
import os
!pwd

/Users/tejasgadi/Local_Disk_D/Machine_Learning_Projects/finetuning_distilbert_text_classification/research


In [4]:
# Goto Root directory of the project
os.chdir("../")
!pwd

/Users/tejasgadi/Local_Disk_D/Machine_Learning_Projects/finetuning_distilbert_text_classification


In [5]:
# Data Ingestion Config Schema/Data Class
from dataclasses import dataclass
from pathlib import Path

@dataclass
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [6]:
from src.textSummarizer.constants import *
from src.textSummarizer.utils.common import read_yaml, create_directories
from box import ConfigBox

## Config Manager

In [7]:
# Create Configuration Manager: This has basic information required before staring any module/components

class ConfigurationManager:
    def __init__(self, config_path = CONFIG_FILE_PATH, params_path = PARAMS_FILE_PATH):
        self.config:ConfigBox = read_yaml(config_path)
        self.params:ConfigBox = read_yaml(params_path)

        # Create Artifacts Root dir
        create_directories([self.config.artifacts_root])
    
    def get_data_transformation_config(self)->DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([config.root_dir])

        data_transformation_config = DataTransformationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            tokenizer_name=config.tokenizer_name
        ) 

        return data_transformation_config


In [8]:
## Data Transfomation operation libraries import

import os
from src.textSummarizer.logging import logger
from transformers import AutoTokenizer 
from datasets import load_from_disk

/Users/tejasgadi/Local_Disk_D/Machine_Learning_Projects/finetuning_distilbert_text_classification/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-04-27 14:28:58,391 : INFO: config: PyTorch version 2.6.0 available.]


## Data Transformation Component

In [17]:
class DataTransformation:
    def __init__(self, data_transformation_config: DataTransformationConfig):
        self.config = data_transformation_config
        self.tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_name)

    # Transforming data into seq to seq model required format for model training
    def convert_examples_to_features(self,example_batch):
        input_encodings = self.tokenizer(example_batch["dialogue"], max_length=1024, truncation=True)

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch["summary"], max_length=1024, truncation=True)
        
        return {
            'input_ids':input_encodings['input_ids'],
            'attention_mask': input_encodings['attention_mask'],
            'labels': target_encodings['input_ids']
        }

    def apply_transformation_to_dataset(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(function=self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))


In [18]:
# Instantiate and call the data transformation methods
config = ConfigurationManager()

data_transformation_config = config.get_data_transformation_config()

data_transformation = DataTransformation(data_transformation_config)

data_transformation.apply_transformation_to_dataset()

[2025-04-27 14:33:53,205 : INFO: common: yaml file: config/config.yaml is loaded sucessfully]
[2025-04-27 14:33:53,206 : INFO: common: yaml file: params.yaml is loaded sucessfully]
[2025-04-27 14:33:53,206 : INFO: common: Created directory at: artifacts]
[2025-04-27 14:33:53,206 : INFO: common: Created directory at: artifacts/data_transformation]


Map:   0%|          | 0/14732 [00:00<?, ? examples/s]/Users/tejasgadi/Local_Disk_D/Machine_Learning_Projects/finetuning_distilbert_text_classification/venv/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:3980: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Saving the dataset (1/1 shards): 100%|██████████| 818/818 [00:00<00:00, 234786.88 examples/s]
